In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath("../../"))
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from src.config import Config
from src.rag import ChromaSlideStore
import pandas as pd

from src.eval import EvaluationConfig, RAGEvaluator
from src.config import load_spreadsheet

from langchain.callbacks import tracing_v2_enabled

config = Config()

/home/groot/Desktop/projects/global/PresRAG/presentation-rag/src/config/model_setup.py:36: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  return ChatOpenAI(


In [3]:
# Enable LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "default"

In [4]:
from ragas.metrics import (
    AnswerCorrectness,
    AnswerRelevancy,
    ContextPrecision,
    ContextRecall,
    Faithfulness,
)

from src.rag.score import (
    ExponentialScorer, HyperbolicScorer, MinScorer,
    ExponentialWeightedScorer, HyperbolicWeightedScorer
)

# Initialize components
embeddings = config.embedding_config.load_vsegpt()
storage = ChromaSlideStore(collection_name="pres0", embedding_model=embeddings)


# Load questions if needed
sheet_id = os.environ["BENCHMARK_SPREADSHEET_ID"]
df_questions = RAGEvaluator.load_questions_from_sheet(sheet_id)
df_questions.sample(5)

,pres_name,question,page,content,comment,MinScorer,Hyperbolic Scorer
9,6.Увеличиваем число обнаружений в задачах_Дмит...,Презентация про компьютерное зрение в которой ...,18,visual,Hyperbolic нашел но другой слайд из этой презы,FAIL,PASS
18,1.За рамками сценария_Эмели Драль_вер.3,В какой презентации обсуждали методы оценки LLM,38,general,NaN,PASS,PASS
8,Kept_Подвижной состав РФ_2024 (20 стр),Про что рассказывал Сергей Казачков?,"2,20",text,NaN,PASS,PASS
24,2.Kolmogorov Arnold Networks_Павел Плюснин_вер.2,Презентация про нейросети Колмогорова-Арнольда,,general,NaN,NaN,NaN
15,AXES_х_Понимаю_Исследование_практик_благополуч...,На слайде были результаты опроса насчет отрасл...,5,visual,NaN,PASS,PASS


In [10]:
dataset_name = "PresRetrieve_25"
# df_eval = df_questions.sample(5)
df_eval = df_questions

eval_config = EvaluationConfig(
    dataset_name=dataset_name,
    ragas_metrics=[],
    scorers=[MinScorer(), ExponentialScorer(), HyperbolicScorer(), ExponentialWeightedScorer(), HyperbolicWeightedScorer()]
)
llm = Config().model_config.load_vsegpt(model="openai/gpt-4o-mini", temperature=0.7)

evaluator = RAGEvaluator(storage=storage, config=eval_config, llm=llm)
evaluator.create_or_load_dataset(df_eval)

Using existing dataset: PresRetrieve_25


Dataset(name='PresRetrieve_25', description=None, data_type=<DataType.kv: 'kv'>, id=UUID('0e7ba37d-c0d5-4ab0-b518-d15880c6059d'), created_at=datetime.datetime(2024, 12, 3, 10, 21, 32, 403478, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 12, 3, 10, 21, 32, 403478, tzinfo=datetime.timezone.utc), example_count=25, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None)

In [ ]:
import nest_asyncio
nest_asyncio.apply()
# Run evaluation
evaluator.run_evaluation()

View the evaluation results for experiment: 'min-982038e6' at:
https://smith.langchain.com/o/44cd8e12-235d-52ea-a8de-1063f25bb9a4/datasets/0e7ba37d-c0d5-4ab0-b518-d15880c6059d/compare?selectedSessions=be6cb0ce-83e1-42a2-8d03-83cd08e836b7




0it [00:00, ?it/s]